In [29]:
import sys
sys.path.append('../../')

from IPython.core.display import display
import numpy as np
import pandas as pd
import networkx as nx
import utils.graph_utils as graph_utils
from utils.IO_utils import load_ground_true
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import svm


In [30]:
# settings
np.random.seed(2020)
name = "Dblp"
dim = 128
come_model_type = "BGMM"
ks = [2, 5, 10, 20]

In [31]:
# import graph and true labels

# graph
G = graph_utils.load_matfile(f"../../data/{name}/{name}.mat", undirected=True)

# labels_true
labels_true, _ = load_ground_true(path=f"../../data/{name}", file_name=name)
labels_true = np.array(labels_true) - 1  # 1..5 -> 0..4

In [32]:
# import node embeddings and predicted labels
dfs = {}
for k in ks:
    # import node embeddings
    df = pd.read_csv(
        f"./data/{name}_alpha-0.1_beta-0.1_ws-10_neg-5_lr-0.025_icom-219_ind-219_ds-0.0_d-{dim}_type-{come_model_type}_k-{k}.txt",
        sep="\t| ",
        header=None
    )
    df = df.rename(columns={0: 'node'})
    df.set_index(['node'], inplace=True)

    # import predicted labels
    labels_pred = pd.read_csv(f"./data/labels_pred_{come_model_type}_d{dim}_k{k}.txt", header=None)
    labels_pred = labels_pred.rename(columns={0: 'label_pred'})
    labels_pred.label_pred = labels_pred.astype(int)

    # join labels pred and true to embeddings
    df = df.join(labels_pred)
    df['label_true'] = labels_true

    print(f"k = {k}")
    display(df)
    display(df.describe())
    dfs[k] = df

/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/Users/anton/minico

k = 2
k = 5
k = 10
k = 20


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
node,,,,,,,,,,,,,,,,,,,,,
0,0.240901,-0.427344,-0.286315,0.765583,0.326247,-0.113467,0.177773,0.388328,-0.029535,0.270559,...,0.029940,-0.129027,-0.562405,-0.204439,-0.521886,0.193663,-0.040719,-0.174307,0,0
1,0.103152,-0.448808,-0.189982,0.734238,0.246045,-0.242816,0.212699,0.372448,-0.172408,0.367357,...,-0.003854,-0.206388,-0.585620,-0.106290,-0.466203,0.092861,0.089877,-0.104623,0,2
2,0.361920,-0.375219,-0.351159,0.767331,0.384430,-0.072904,0.084759,0.361776,0.075714,0.260104,...,-0.027313,-0.029863,-0.552728,-0.259638,-0.499772,0.107687,0.045251,-0.179297,0,2
3,0.268073,-0.369202,-0.312004,0.702251,0.270890,-0.203112,0.135949,0.380749,0.027111,0.294283,...,-0.029398,-0.108868,-0.535339,-0.181532,-0.342876,0.175863,-0.027251,-0.280337,0,0
4,0.193797,-0.303130,-0.325424,0.712492,0.328340,-0.128894,-0.019049,0.412264,0.087277,0.311296,...,0.054580,-0.116861,-0.621419,-0.253226,-0.397189,0.120723,-0.154705,-0.282843,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.148476,-0.308252,-0.139955,0.611333,0.383165,-0.146370,0.044713,0.401092,0.050012,0.289585,...,-0.169100,-0.036170,-0.515795,-0.255293,-0.147932,0.170850,0.064360,-0.201557,0,0
13180,-0.162514,-0.574927,-0.308982,-0.072643,0.458607,0.089006,0.096566,0.060951,0.308874,0.386778,...,-0.248846,0.341997,0.104415,0.181790,0.102692,-0.378308,0.130314,0.329824,1,3
13181,-0.229493,-0.479934,-0.079216,0.005251,0.405667,-0.047267,0.042990,-0.112940,0.404113,0.475717,...,-0.042491,0.333508,-0.005756,0.105983,-0.037435,-0.437259,0.077030,0.366979,1,3


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
count,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,...,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000
mean,-0.060061,-0.274079,-0.020643,0.282961,0.313812,-0.140786,0.137279,0.245693,0.110701,0.397177,...,-0.134649,0.247830,-0.267642,-0.018710,-0.074114,0.026915,0.137623,0.157681,0.587227,2.190382
std,0.167313,0.210718,0.161967,0.231464,0.150630,0.184355,0.145522,0.163225,0.159814,0.150229,...,0.151159,0.152502,0.247313,0.148347,0.157160,0.190498,0.176870,0.256473,0.492351,1.315339
min,-0.704691,-0.923463,-0.531326,-0.403197,-0.245462,-0.695142,-0.408513,-0.469149,-0.445943,-0.324243,...,-0.660680,-0.352469,-0.966586,-0.673103,-0.661598,-0.606917,-0.471190,-0.633908,0.000000,0.000000
25%,-0.166515,-0.432207,-0.127438,0.108042,0.214475,-0.276543,0.044748,0.135052,-0.006175,0.304504,...,-0.235069,0.146563,-0.485280,-0.108572,-0.179612,-0.102121,0.015811,-0.072808,0.000000,1.000000
50%,-0.050948,-0.273596,-0.029817,0.279143,0.316315,-0.151441,0.137749,0.243988,0.107979,0.403107,...,-0.130263,0.252237,-0.254176,-0.019499,-0.067489,0.016870,0.116913,0.185152,1.000000,3.000000
75%,0.053388,-0.117520,0.082956,0.470545,0.412722,-0.006481,0.233837,0.353649,0.232909,0.496245,...,-0.032083,0.352962,-0.059609,0.073818,0.037277,0.165648,0.246646,0.365344,1.000000,3.000000
max,0.501491,0.432512,0.645468,0.999915,0.907505,0.466024,0.633653,0.889513,0.638559,0.900169,...,0.426193,0.793886,0.399098,0.576733,0.390375,0.786175,0.812152,0.849675,1.000000,4.000000


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
node,,,,,,,,,,,,,,,,,,,,,
0,0.220834,-0.553853,-0.328899,0.671641,0.329924,0.173045,-0.102815,0.285762,0.003380,0.112134,...,0.182161,-0.278452,-0.486919,-0.130558,-0.587542,0.073183,0.018717,-0.162010,0,0
1,0.053342,-0.540413,-0.219000,0.667815,0.263702,-0.052674,0.027995,0.341619,-0.178470,0.226588,...,0.103897,-0.446885,-0.420519,-0.017391,-0.528854,-0.068375,0.154689,-0.071276,0,2
2,0.322656,-0.497658,-0.473506,0.719183,0.355759,0.095999,-0.134738,0.320118,0.163740,0.274847,...,0.087212,-0.180086,-0.403072,-0.249232,-0.510452,-0.017414,0.081143,-0.145351,0,2
3,0.323484,-0.559512,-0.282649,0.562199,0.253566,-0.024401,-0.020219,0.298419,0.079013,0.191254,...,0.126351,-0.349942,-0.302908,-0.235230,-0.407187,0.040414,-0.039978,-0.286869,0,0
4,0.191777,-0.525736,-0.342277,0.553256,0.246524,0.114857,-0.274811,0.309242,0.148813,0.139341,...,0.119310,-0.279968,-0.476109,-0.211932,-0.451443,0.028797,-0.097451,-0.337593,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.175474,-0.440159,-0.176025,0.489979,0.358946,0.043824,-0.086921,0.311323,0.093141,0.207125,...,-0.082558,-0.272513,-0.339147,-0.278023,-0.229072,-0.078072,0.164565,-0.001591,0,0
13180,-0.131393,-0.610317,-0.269538,-0.101288,0.458916,0.044181,0.120444,-0.011951,0.372034,0.423370,...,-0.279484,0.313747,0.178657,0.116869,0.139234,-0.371700,0.145553,0.353853,3,3
13181,-0.149912,-0.519458,-0.078247,-0.004047,0.463817,-0.076020,0.061312,-0.161032,0.455968,0.474561,...,-0.055181,0.286756,0.107003,0.057914,-0.048719,-0.420546,0.075961,0.430896,3,3


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
count,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,...,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000
mean,-0.039701,-0.265758,-0.023334,0.228664,0.270458,-0.101458,0.088516,0.220926,0.088744,0.362758,...,-0.084892,0.166022,-0.187358,-0.007592,-0.073773,-0.011505,0.128060,0.178253,1.968447,2.190382
std,0.175732,0.222564,0.194844,0.210735,0.173370,0.235491,0.180035,0.183207,0.195180,0.171931,...,0.163302,0.189321,0.221935,0.158285,0.177741,0.189716,0.190853,0.230748,1.318126,1.315339
min,-0.750939,-0.965408,-0.650093,-0.451433,-0.453869,-0.717138,-0.563422,-0.499279,-0.518311,-0.539191,...,-0.753671,-0.641493,-0.899814,-0.737821,-0.766114,-0.664359,-0.477902,-0.608761,0.000000,0.000000
25%,-0.147778,-0.436675,-0.153692,0.077509,0.157040,-0.275795,-0.026141,0.102212,-0.058191,0.260401,...,-0.189010,0.042763,-0.349771,-0.102534,-0.197773,-0.129228,-0.004635,0.000277,1.000000,1.000000
50%,-0.036864,-0.267714,-0.027340,0.228208,0.271650,-0.128508,0.092284,0.211780,0.082110,0.371315,...,-0.071919,0.175500,-0.182568,-0.009010,-0.074789,-0.003466,0.113862,0.172414,2.000000,3.000000
75%,0.076353,-0.101885,0.102134,0.381674,0.385558,0.067745,0.211559,0.332262,0.229038,0.476590,...,0.024088,0.298939,-0.023515,0.085801,0.051445,0.114198,0.241524,0.349823,3.000000,3.000000
max,0.580030,0.501283,0.725406,0.938984,0.937811,0.690536,0.762150,0.956331,0.764427,0.992371,...,0.463942,0.850441,0.485664,0.684267,0.553291,0.727348,0.858526,0.911136,4.000000,4.000000


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
node,,,,,,,,,,,,,,,,,,,,,
0,0.286513,-0.539440,-0.324212,0.612531,0.220063,0.015704,0.129525,0.370951,-0.056892,0.164724,...,0.047913,-0.246448,-0.467069,-0.257212,-0.460152,-0.023907,0.000885,-0.216241,5,0
1,0.107634,-0.509803,-0.209078,0.611501,0.201776,-0.065335,0.096588,0.372340,-0.317339,0.342580,...,-0.022664,-0.368592,-0.497054,-0.181161,-0.508367,-0.196089,0.210439,-0.012135,5,2
2,0.392686,-0.467933,-0.439336,0.593243,0.282720,-0.005257,-0.057860,0.386462,0.065102,0.167917,...,-0.016327,-0.187759,-0.445138,-0.319251,-0.508681,-0.129692,0.070145,-0.216227,5,2
3,0.348014,-0.524133,-0.296802,0.487814,0.169853,-0.123029,0.078116,0.334139,0.019137,0.266734,...,0.037562,-0.261748,-0.389569,-0.296537,-0.368414,-0.056314,0.003230,-0.334724,5,0
4,0.198726,-0.433397,-0.320578,0.553522,0.184439,-0.010608,-0.137308,0.340164,0.076889,0.184239,...,0.083498,-0.229865,-0.458667,-0.281375,-0.384260,-0.039693,-0.130239,-0.394700,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.226512,-0.398366,-0.233069,0.402589,0.247345,-0.085590,-0.136896,0.437692,0.045567,0.291592,...,-0.088294,-0.158597,-0.313603,-0.326902,-0.222201,-0.174680,0.240955,-0.147498,5,0
13180,-0.085753,-0.548645,-0.352943,-0.012771,0.456612,0.185792,0.011847,-0.069382,0.299814,0.587432,...,-0.145604,0.297858,0.045936,0.166874,0.289517,-0.580532,-0.066596,0.054021,1,3
13181,-0.139509,-0.421295,-0.190604,0.046272,0.425104,0.003923,-0.139175,-0.077919,0.419698,0.531760,...,-0.108737,0.139637,-0.119146,0.078456,0.117390,-0.565436,-0.045055,0.009875,1,3


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
count,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,...,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000
mean,-0.010577,-0.264898,-0.023454,0.215537,0.288917,-0.114644,0.096188,0.214145,0.066704,0.364629,...,-0.098843,0.162721,-0.196499,-0.026878,-0.056160,-0.027049,0.116083,0.138241,3.647376,2.190382
std,0.190622,0.225915,0.207956,0.214910,0.180803,0.229866,0.184376,0.187921,0.201252,0.179247,...,0.193567,0.198533,0.208579,0.177258,0.184465,0.181977,0.201884,0.215939,2.694397,1.315339
min,-0.772900,-1.068519,-0.679043,-0.529304,-0.466229,-0.807965,-0.629157,-0.465883,-0.581136,-0.521377,...,-0.812310,-0.624352,-0.967678,-0.785453,-0.746426,-0.666557,-0.542644,-0.645040,0.000000,0.000000
25%,-0.132298,-0.429510,-0.164935,0.061436,0.173148,-0.280200,-0.028370,0.087814,-0.081521,0.259968,...,-0.219472,0.029638,-0.336901,-0.134046,-0.183372,-0.142778,-0.025322,-0.024471,1.000000,1.000000
50%,-0.008846,-0.263051,-0.030782,0.218065,0.288168,-0.130732,0.095953,0.205825,0.058446,0.375374,...,-0.082918,0.168345,-0.189210,-0.025762,-0.060049,-0.023318,0.100803,0.133432,3.000000,3.000000
75%,0.116560,-0.098990,0.115513,0.370231,0.408952,0.045499,0.222129,0.334495,0.210195,0.484757,...,0.038112,0.302096,-0.052524,0.084398,0.074501,0.091139,0.235993,0.299221,5.000000,3.000000
max,0.791898,0.490275,0.743585,0.989255,0.986581,0.638238,0.723775,0.897275,0.763974,1.010176,...,0.557327,0.952594,0.490575,0.657917,0.574697,0.743633,0.908643,0.826210,9.000000,4.000000


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
node,,,,,,,,,,,,,,,,,,,,,
0,0.198849,-0.564162,-0.365433,0.674886,0.184291,0.007714,0.184088,0.342207,0.012706,0.118765,...,0.110789,-0.273289,-0.526778,-0.107955,-0.495944,-0.038982,0.056455,-0.180739,14,0
1,0.072871,-0.528823,-0.237065,0.762957,0.115731,-0.038232,0.236198,0.337882,-0.280086,0.207977,...,-0.028891,-0.381430,-0.610548,0.042456,-0.555906,-0.181692,0.272336,-0.074232,14,2
2,0.359639,-0.581496,-0.502440,0.572398,0.217842,0.098364,-0.001944,0.309491,0.175916,0.135551,...,0.049407,-0.182300,-0.507461,-0.210184,-0.531672,-0.096870,0.112191,-0.188631,14,2
3,0.259662,-0.498589,-0.393288,0.614545,0.188136,-0.073070,0.179199,0.370779,0.008329,0.096162,...,0.039356,-0.260061,-0.426436,-0.146911,-0.312892,-0.033156,0.056913,-0.275924,14,0
4,0.103670,-0.419906,-0.454309,0.575908,0.121455,0.091497,-0.005476,0.327171,0.150284,0.172452,...,0.147098,-0.328749,-0.474680,-0.186721,-0.344077,-0.026667,-0.028824,-0.180251,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.228699,-0.273552,-0.176484,0.559736,0.170294,-0.095328,0.108752,0.281892,-0.067520,0.240036,...,-0.120002,-0.250101,-0.416892,-0.181314,-0.240955,-0.302917,0.371771,-0.117189,14,0
13180,-0.076988,-0.565519,-0.167976,-0.042378,0.331466,0.079318,0.211200,-0.142228,0.381507,0.437791,...,-0.253791,0.316220,0.071240,0.183490,0.117619,-0.460454,0.119785,0.272065,19,3
13181,-0.066074,-0.504302,0.007374,-0.010019,0.292341,0.012342,0.165189,-0.215115,0.461236,0.535412,...,-0.027444,0.315652,0.063959,0.185397,0.011933,-0.482092,0.090645,0.339462,19,3


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
count,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,...,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000
mean,-0.018753,-0.248977,-0.022163,0.251637,0.267426,-0.122886,0.120077,0.179475,0.089719,0.354926,...,-0.111742,0.177136,-0.190998,-0.007011,-0.058163,-0.021602,0.114051,0.149616,10.459041,2.190382
std,0.206252,0.224612,0.205772,0.220158,0.198323,0.229413,0.190302,0.204879,0.182197,0.187090,...,0.190724,0.204034,0.200655,0.172182,0.197508,0.187073,0.199140,0.214780,6.194439,1.315339
min,-0.833450,-0.920029,-0.705258,-0.440527,-0.375317,-0.763843,-0.598295,-0.601708,-0.504951,-0.428983,...,-0.826163,-0.582752,-0.887583,-0.691189,-0.768463,-0.737050,-0.586415,-0.674043,0.000000,0.000000
25%,-0.147791,-0.413751,-0.163148,0.092049,0.134387,-0.291999,-0.006754,0.036340,-0.040800,0.234564,...,-0.236721,0.038633,-0.329133,-0.115797,-0.198458,-0.141645,-0.023508,-0.009145,6.000000,1.000000
50%,-0.011238,-0.247077,-0.028939,0.256009,0.262872,-0.141927,0.119651,0.170131,0.088061,0.357754,...,-0.101397,0.181634,-0.194490,-0.004778,-0.053244,-0.016564,0.100746,0.143935,11.000000,3.000000
75%,0.122249,-0.084242,0.112149,0.413492,0.404691,0.031605,0.251222,0.312096,0.221984,0.477183,...,0.018508,0.318330,-0.055419,0.102589,0.080621,0.100290,0.233500,0.307119,16.000000,3.000000
max,0.694369,0.556471,0.709315,1.033625,1.036752,0.659143,0.731574,0.931766,0.744499,1.003484,...,0.506263,0.871657,0.443020,0.637680,0.574418,0.778035,0.856380,0.845602,19.000000,4.000000


# Community Detection

In [33]:
# Conductance

for k in dfs:
    print("K =", k)
    df = dfs[k]
    conductance_min = 1
    for i in range(k):
        nodes = df[df.label_pred == i].index.to_numpy()
        if len(nodes) == 0: continue
        conductance_i = nx.conductance(G, nodes)
        print(f"  i{i} conductance: {conductance_i}")
        conductance_min = min(conductance_min, conductance_i)
    print(f"=>conductance: {conductance_min}")

K = 2
  i0 conductance: 0.05153027478804642
  i1 conductance: 0.05153027478804642
=>conductance: 0.05153027478804642
K = 5
  i0 conductance: 0.06110668789808917
  i1 conductance: 0.20179047791291765
  i2 conductance: 0.011492883939736796
  i3 conductance: 0.10103341025383766
  i4 conductance: 0.12538141470180306
=>conductance: 0.011492883939736796
K = 10
  i0 conductance: 0.23366860772895712
  i1 conductance: 0.35628851032982967
  i2 conductance: 0.4151607603848862
  i3 conductance: 0.17851810377006347
  i4 conductance: 0.04919872794163497
  i5 conductance: 0.11582060128141942
  i6 conductance: 0.4380487804878049
  i7 conductance: 0.10034728568817401
  i8 conductance: 0.43238993710691825
  i9 conductance: 0.47928107838242634
=>conductance: 0.04919872794163497
K = 20
  i0 conductance: 0.47514619883040937
  i1 conductance: 0.5147149619263223
  i2 conductance: 0.43024618991793667
  i3 conductance: 0.4318021201413428
  i4 conductance: 0.4680914217868804
  i5 conductance: 0.45426114151681
 

In [34]:
# NMI

for k in dfs:
    print("K =", k)
    df = dfs[k]
    labels_pred = df.label_pred.to_numpy()
    labels_true = df.label_true.to_numpy()

    nmi = metrics.normalized_mutual_info_score(labels_true, labels_pred)
    print("NMI: ", nmi)

K = 2
NMI:  0.43575239496704926
K = 5
NMI:  0.668553614850714
K = 10
NMI:  0.47369605057376785
K = 20
NMI:  0.30816578800908423


# Node Classification

In [35]:
# train LibSVM classifier on 90% of data and test on 10%.

for k in ks:
    print(f"k = {k}")
    df = dfs[k]

    # train/test split
    train, test = train_test_split(df, test_size=0.1)
    feature_col = np.arange(0, dim) + 1
    X_train = train.loc[:, feature_col]
    Y_train = train.label_true
    X_test = test.loc[:, feature_col]
    Y_test = test.label_true

    # fit
    clf = svm.SVC()
    clf.fit(X_train, Y_train)

    # pred
    Y_pred = clf.predict(X_test)

    # results
    micro_f1 = metrics.f1_score(Y_test, Y_pred, average='micro')
    macro_f1 = metrics.f1_score(Y_test, Y_pred, average='macro')
    print(f"  micro_f1: {micro_f1*100:.1f}")
    print(f"  macro_f1: {macro_f1*100:.1f}")

k = 2
  micro_f1: 92.6
  macro_f1: 92.7
k = 5
  micro_f1: 92.6
  macro_f1: 92.4
k = 10
  micro_f1: 93.3
  macro_f1: 93.0
k = 20
  micro_f1: 92.3
  macro_f1: 91.9
